In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import MobileNetV2, MobileNet, NASNetMobile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# 設置數據生成器
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    validation_split=0.2
)

# 訓練數據生成器
train_generator = train_datagen.flow_from_directory(
    r'E:\Codes\CV train\回收系統dataset\garbage',  
    target_size=(224, 224),  
    batch_size=16,
    class_mode='categorical',
    subset='training'  # 指定為訓練集
)

# 驗證數據生成器
validation_generator = train_datagen.flow_from_directory(
    r'E:\Codes\CV train\回收系統dataset\garbage',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'  # 指定為驗證集
)

Found 6465 images belonging to 4 classes.
Found 1614 images belonging to 4 classes.


In [4]:
# 加載預訓練模型
base_model = MobileNetV2(
    #alpha=1.3,
    weights='imagenet', 
    include_top=False, 
    input_shape=(224, 224, 3))

# 添加自定義分類層
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dense(4, activation='softmax')(x)

# 構建完整模型
model = Model(inputs=base_model.input, outputs=x)

# 解凍預訓練層
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[:4]:  
    layer.trainable = True

    
# 編譯模型
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [5]:
checkpoint = ModelCheckpoint(
    "test.h5",   # 保存的文件名稱
    monitor="val_accuracy",  # 監測驗證準確率（也可以改成 "val_loss"）
    save_best_only=True,     # 只儲存最佳權重
    mode="max",              # "max" 代表數值越高越好（適用於準確率）
    verbose=1                # 顯示存檔訊息
)

In [6]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    steps_per_epoch=train_generator.samples // 16,
    validation_steps=validation_generator.samples // 16,
    callbacks=[checkpoint]  
)

Epoch 1/100
404/404 [==============================] - 53s 126ms/step - loss: 0.3777 - accuracy: 0.8552 - val_loss: 0.3938 - val_accuracy: 0.8619

Epoch 00001: val_accuracy improved from -inf to 0.86187, saving model to test.h5
Epoch 2/100


c:\Users\willy\anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


404/404 [==============================] - 50s 125ms/step - loss: 0.2563 - accuracy: 0.9049 - val_loss: 0.4099 - val_accuracy: 0.8394

Epoch 00002: val_accuracy did not improve from 0.86187
Epoch 3/100
404/404 [==============================] - 50s 124ms/step - loss: 0.2195 - accuracy: 0.9170 - val_loss: 0.4709 - val_accuracy: 0.8475

Epoch 00003: val_accuracy did not improve from 0.86187
Epoch 4/100
404/404 [==============================] - 50s 124ms/step - loss: 0.2076 - accuracy: 0.9203 - val_loss: 0.3397 - val_accuracy: 0.8888

Epoch 00004: val_accuracy improved from 0.86187 to 0.88875, saving model to test.h5
Epoch 5/100
404/404 [==============================] - 50s 123ms/step - loss: 0.1863 - accuracy: 0.9262 - val_loss: 0.2921 - val_accuracy: 0.8969

Epoch 00005: val_accuracy improved from 0.88875 to 0.89688, saving model to test.h5
Epoch 6/100
404/404 [==============================] - 51s 125ms/step - loss: 0.1688 - accuracy: 0.9384 - val_loss: 0.2962 - val_accuracy: 0.8956


In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________